In [1]:
import os

import torch

os.sys.path.append("/datadrive5/huypn16/ICV")

from models import build_tokenizer, build_model
from tasks import load_task
from utils.llm_layers import add_icv_layers, remove_icv_layers

from vllm import EngineArgs, LLMEngine, SamplingParams
from vllm.control_vectors.request import ControlVectorRequest
from transformers import AutoTokenizer
import numpy as np
torch.cuda.is_available()

/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-06 06:40:10,972	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


True

In [2]:
MODEL_PATH = "Qwen/Qwen2.5-7B-Instruct"
engine_args = EngineArgs(model=MODEL_PATH, enable_control_vector=True, gpu_memory_utilization=0.95, return_hidden_states=True)
engine = LLMEngine.from_engine_args(engine_args)

INFO 10-06 06:40:12 llm_engine.py:200] Initializing an LLM engine (v0.5.5) with config: model='Qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, return_hidden_states=True, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-7B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_output_pr

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.21it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.87it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.75it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.75it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.79it/s]



INFO 10-06 06:40:16 model_runner.py:920] Loading model weights took 14.2487 GB
INFO 10-06 06:40:20 gpu_executor.py:123] # GPU blocks: 57734, # CPU blocks: 4681
INFO 10-06 06:40:25 model_runner.py:1239] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-06 06:40:25 model_runner.py:1243] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-06 06:40:37 model_runner.py:1358] Graph capturing finished in 13 secs.


In [ ]:
cv_path = "/datadrive5/huypn16/ICV/controlvector.gguf"


def generate_n_get_icv(bases, engine, max_tokens=2000):
    prompts = [
        (base, SamplingParams(temperature=0.0, max_tokens=max_tokens), None) for base in bases
    ]
    request_id = 0
    results = set()
    
    icvs = {}
    
    # icv_pair = {"base": None, "next": None}
    
    while prompts or engine.has_unfinished_requests():
        if prompts:
            prompt, sampling_params, cv_request = prompts.pop(0)
            engine.add_request(str(request_id), prompt, sampling_params, control_vector_request=cv_request)
            request_id += 1

        request_outputs = engine.step()
        for request_output in request_outputs:
            if request_output.finished or request_output.outputs[0].prompt_hidden_states != None:
                results.add((request_output.request_id, request_output.outputs[0].text))
                if icvs.get(request_output.request_id) == None:
                    icvs[request_output.request_id] = {"base": None, "next": None}
                if request_output.outputs[0].prompt_hidden_states != None:
                    icvs[request_output.request_id]["base"] = request_output.outputs[0].prompt_hidden_states
                icvs[request_output.request_id]["next"] = request_output.outputs[0].hidden_states                  
        if len(results) == len(bases):
            break
    
    ordered_requests = sorted(results, key=lambda x: int(x[0]))
    ordered_icvs = sorted(icvs.items(), key=lambda x: int(x[0]))
    
    for icv in ordered_icvs:
        icv[1]["difference"] = icv[1]["next"] - icv[1]["base"]
    
    final_output = [(req[1], icv[1]["difference"]) for req, icv in zip(ordered_requests, ordered_icvs)]
    return final_output
    
bases = ["Solving the following mathematical problem. Problem: Calculate the following expression: (10222 + 23123123 * 4 - 1 ) * 5 - 6. Step 1:" ]

output = generate_n_get_icv(bases, engine, max_tokens=2000)
print(output)

# Task 1: Dialogue safety

In [5]:
prefix = """
Problem:  Let $(a_1,b_1),$ $(a_2,b_2),$ $\dots,$ $(a_n,b_n)$ be all the ordered pairs $(a,b)$ of complex numbers with $a^2+b^2\\neq 0,$
\[a+\\frac{10b}{a^2+b^2}=5, \quad \\text{and} \quad b+\\frac{10a}{a^2+b^2}=4.\]Find $a_1 + b_1 + a_2 + b_2 + \dots + a_n + b_n.$ 

#### Step 1: If $a = 0,$ then $\\frac{10}{b} = 5,$ so $b = 2,$ which does not satisfy the second equation.

#### Step 2: If $b = 0,$ then $\\frac{10}{a} = 4,$ so $a = \\frac{5}{2},$ which does not satisfy the first equation.

#### Step 3: So, we can assume that both $a$ and $b$ are nonzero.

#### Step 4: Then $\\frac{5 - a}{b} = \\frac{4 - b}{a} = \\frac{10}{a^2 + b^2}.$

#### Step 5: \[\\frac{5b - ab}{b^2} = \\frac{4a - ab}{a^2} = \\frac{10}{a^2 + b^2},\]so
\[\\frac{4a + 5b - 2ab}{a^2 + b^2} = \\frac{10}{a^2 + b^2},\]so $4a + 5b - 2ab = 10.$

#### Step 6: Then $2ab - 4a - 5b + 10 = 0,$ which factors as $(2a - 5)(b - 2) = 0.$  Hence, $a = \\frac{5}{2}$ or $b = 2.$"""

suffix = """#### Step 7: If $a = \\frac{5}{2},$ then \[\\frac{5/2}{b} = \\frac{10}{\\frac{25}{4} + b^2}.\]. This simplifies to $4b^2 - 16b + 25 = 0.$  By the quadratic formula,
\[b = 2 \pm \\frac{3i}{2}.\]"""
math_directions = [(prefix + suffix, prefix + """#### Step 7: If $a = \\frac{5}{2},$ then
\[\\frac{5}{2} + \\frac{20b}{\\frac{25}{4} + b^2} = 5,\]so $\\frac{20b}{\\frac{25}{4} + b^2} = \\frac{5}{2},$ so $\\frac{b}{\\frac{5}{4} + b^2} = \\frac{1}{4},$ so $4b = \\frac{5}{4} + b^2,$ so $b^2 - 4b + \\frac{5}{4} = 0.$"""),
                   (prefix + suffix, prefix + "#### Step 7: If $a = \\frac{5}{2},$ then $\\frac{10}{b} = 5,$ so $b = 2.$"),
                   (prefix + suffix, prefix + """#### Step 7: If $a = \\frac{5}{2},$ then $\\frac{5}{2} + \\frac{10b}{\\frac{25}{4} + b^2} = 5.$  Letting $k = b^2,$ we get"""),
                   (prefix + suffix, prefix + """#### Step 7: If $a = \\frac{5}{2},$ then
\begin{align*}
\frac{10}{a^2 + b^2} &= 4, \\
\frac{10}{\frac{25}{4} + b^2} &= 4, \\
10 &= 4 \left( \frac{25}{4} + b^2 \right), \\
10 &= 25 + 4b^2, \\
4b^2 &= -15,
\end{align*}which is impossible.""")]
rewards = [0.5927, 0.5312, 0.4688, 0.5]

In [6]:
prefix = """Solving the following mathematical problem
Problem: Calculate the following expression: (4 + 3 * 2 - 1 ) * 4 - 2.
"""

suffix = """Step 1: First, we add 4 and 3 to get 7."""

math_directions = [(prefix + suffix, prefix + "Step 1: First, we minus 1 from 2 to get 1."), # 0
                   (prefix + suffix, prefix + "Step 1: First, we multiply 3 by 2 to get 6."), # 0.7
                   (prefix + suffix, prefix + "Step 1: First, we minus 2 from 4 to get 2."), # 0
                   (prefix + suffix, prefix + "Step 1: First, we calculate the expression inside the parentheses to get 3*2 = 6, and then we add 4 to get 10.")] # 1
rewards = [1, 0, 1, 0]

In [7]:
directions = ["Solving the following mathematical problem. Problem: Calculate the following expression: (4 + 3 * 2 - 1 ) * 4 - 2. Step 1: First, we add 4 and 3 to get 7.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 23 * 4 - 1 ) * 3 - 2. Step 1: First, we add 12 and 23 to get 35.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (34 + 2231 * 6 - 1 ) * 34 - 2. Step 1: First, we add 34 and 2231 to get 2265.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 123 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 123 to get 135.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12445 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 12445 to get 12457.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12446 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 123 to get 12458.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12447 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 123 to get 12459.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 1000 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 1000 to get 1012.",]


basesteps = ["Solving the following mathematical problem. Problem: Calculate the following expression: (4 + 3 * 2 - 1 ) * 4 - 3111.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 23 * 4 - 1 ) * 3 - 1245.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (34 + 2231 * 6 - 1 ) * 34 - 123.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 123 * 2 - 1 ) * 412 - 123.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12445 * 2 - 1 ) * 412 - 2.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12446 * 2 - 1 ) * 412 - 2.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12447 * 2 - 1 ) * 412 - 2.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 1000 * 2 - 1 ) * 412 - 2.",]

# math_directions = [(prefix + suffix, prefix + "Step 1: First, we minus 1 from 2 to get 1."), # 0
#                    (prefix + suffix, prefix + "Step 1: First, we multiply 3 by 2 to get 6."), # 0.7
#                    (prefix + suffix, prefix + "Step 1: First, we minus 2 from 4 to get 2."), # 0
#                    (prefix + suffix, prefix + "Step 1: First, we calculate the expression inside the parentheses to get 3*2 = 6, and then we add 4 to get 10.")] # 1
# rewards = [1, 0, 1, 0]
rewards = [1, 0.881212, 0.78841, 0.945423, 0.8, 0.9, 0.95, 0.9]
math_directions = [(basestep, direction) for basestep, direction in zip(basesteps, directions)]
prefix = "Solving the following mathematical problem. Problem: Calculate the following expression: (10222 + 23123123 * 4 - 1 ) * 5 - 6. Step 1:"

In [ ]:
n_comps = 8
icv_reward_weighed = task_agent.get_reward(math_directions, rewards=rewards, rank=n_comps
        )

icv_unweighted, _ = task_agent.obtain_icv(
            math_directions, tokenizer, prefix=("", "")
            ), rank=n_comps
        )
print(len(icv_reward_weighed))
print(len(icv_unweighted))
print(icv_reward_weighed[0].shape)
print(icv_unweighted[0].shape)

query = tokenizer(prefix)

for i in range(n_comps):
    weighed_direction = icv_reward_weighed[i][1:]
    unweighted_direction = icv_unweighted[i][1:]

    weighed_direction = [weighed_direction]
    unweighted_direction = [unweighted_direction]
    lam = 0.2
    print(f"===================={i}th direction===============")
    add_icv_layers(model, torch.stack(unweighted_direction,dim=1).cuda(), [lam])
    

# Original model (Unsafe) 

In [ ]:
generation_output = model.generate(
                        input_ids=torch.tensor(query['input_ids']).unsqueeze(0).cuda(),
                        attention_mask=torch.tensor(query['attention_mask']).unsqueeze(0).cuda(),
                        max_new_tokens=100,
                        temperature = 0.65,
                        do_sample=True,
                        top_k=10,
                        num_return_sequences=1,
                        eos_token_id=[198, tokenizer.eos_token_id]
                    )
print(generation_output)
decoded_output = tokenizer.decode(generation_output[0])
print(decoded_output)

In [ ]:
import gguf
def export_gguf(path: os.PathLike[str] | str, directions: list[torch.Tensor], model):
    arch = "controlvector"
    directions = [directions[i] for i in range(1, len(directions))]
    writer = gguf.GGUFWriter(path, arch)
    writer.add_string(f"{arch}.model_hint", model.config.model_type)
    writer.add_uint32(f"{arch}.layer_count", len(directions))
    for layer, _ in enumerate(directions):
        writer.add_tensor(f"direction.{layer}", directions[layer].numpy())
    writer.write_header_to_file()
    writer.write_kv_data_to_file()
    writer.write_tensors_to_file()
    writer.close()
    
export_gguf("controlvector.gguf", icv_unweighted[6], model)